<a href="https://colab.research.google.com/github/Yasaman-habibi/Pre_Processing_Report/blob/main/Word_Cloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wordcloud

In [ ]:
#Import Library

import os
import re
import nltk
import glob
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive, files
from google.colab import drive
import ipywidgets as widgets
import seaborn as sns
import numpy as np
from wordcloud import WordCloud

In [ ]:
#Upload Files

drive.mount('/content/drive')
source_path = "/content/drive/MyDrive/Combined_Texts"
all_txt_files = glob.glob(os.path.join(source_path, "*.txt"))

selector = widgets.SelectMultiple(
    options=all_txt_files,
    description='Select files',
    rows=10
)
display(selector)

In [ ]:
uploaded_Texts = list(selector.value)
print(" انتخاب شد:\n" + "\n".join(uploaded_Texts))

In [ ]:
#Function

def process_reports_tagged_and_wordcloud(uploaded_Texts, sustain_terms, wordcloud_path):
    records = []

    sustain_terms = sorted(set([kw.strip().lower() for kw in sustain_terms]),
                           key=lambda x: len(x.split()), reverse=True)

    for file_path in uploaded_Texts:
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()

        pattern = r"===== Start of File: (.+?) =====\n(.*?)\n===== End of File: \1 ====="
        reports = re.findall(pattern, content, flags=re.DOTALL)

        for report_name, report_text in reports:
            text = report_text.lower()
            counts = {}

            for term in sustain_terms:
                term_pattern = r"\b" + re.escape(term) + r"\b"
                matches = list(re.finditer(term_pattern, text))
                if matches:
                    counts[term] = len(matches)
                    text = re.sub(term_pattern, " " * len(term), text)

            for kw, cnt in counts.items():
                records.append({"report_name": report_name, "keyword": kw, "count": cnt})

    df = pd.DataFrame(records)

    df["report_name"] = df["report_name"].str.replace(r"\s*\(\d+\)$", "", regex=True)
    df = df.drop_duplicates()

    os.makedirs(wordcloud_path, exist_ok=True)

    # Create Function
    def create_wordcloud_from_freq(freq_dict, title, save_path):
        wc = WordCloud(width=1600, height=1200,background_color="white",colormap="viridis",max_words=150,min_font_size=14)

        wc.generate_from_frequencies(freq_dict)
        plt.figure(figsize=(10,7))
        plt.imshow(wc, interpolation="bilinear")
        plt.axis("off")
        plt.title(title, fontsize=18)
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.show()


    # WordCloud
    total_freq = df.groupby("keyword")["count"].sum().to_dict()
    create_wordcloud_from_freq(total_freq, "All Reports", os.path.join(wordcloud_path, "wordcloud_all_keywords.png"))


    # Word cloud year by year
    df_reset = df.reset_index(drop=True)
    df_reset['year'] = df_reset['report_name'].str[:4]
    years = df_reset['year'].unique()

    for year in years:
        df_year = df_reset[df_reset['year'] == year]
        freq = df_year.groupby("keyword")["count"].sum().to_dict()
        create_wordcloud_from_freq(freq, f"Word Cloud for Year {year}",
                                   os.path.join(wordcloud_path, f"wordcloud_keywords_{year}.png"))

    print(" وردکلادها ساخته و ذخیره شدند.")
    return df

In [ ]:
sustain_Dic = pd.read_excel("/content/drive/MyDrive/sustainability_table/PivotReport/keywords.xlsx")
keywords = sustain_Dic["keyword"].dropna().str.strip().tolist()

wordcloud_path = "/content/drive/MyDrive/sustainability_table/wordclouds"
os.makedirs(wordcloud_path, exist_ok=True)

In [ ]:
pivot_df = process_reports_tagged_and_wordcloud(uploaded_Texts, keywords , wordcloud_path)